In [2]:
# Packages used in this chapter
# To check for installed packages in V1.0 we now need to use the PKG API

using Pkg
ipks = Pkg.installed()
pkgs = ["BenchmarkTools","MacroTools", "Lazy"]
for p in pkgs
    !haskey(ipks,p) && Pkg.add(p)
end

# Chapter 04

In [3]:
# We can define these here or as required
using BenchmarkTools, MacroTools, Lazy, PyPlot

# We need these as some functions have moved from Base to Stdlib
#
using Printf, Statistics, SpecialFunctions

In [4]:
# We can define these here or as required
using BenchmarkTools, MacroTools, PyPlot

# We need these as some functions have moved from Base to Stdlib
#
using Printf, Statistics


## Multiple Dispatch

In [5]:
recip(x::Number) = 
(x == zero(typeof(x))) ? error("Invalid reciprocal") : one(typeof(x)) / x

recip (generic function with 1 method)

In [6]:
recip(2)

0.5

In [7]:
recip(recip(2))

2.0

In [8]:
recip(11//17)

17//11

In [9]:
recip(11 + 17im)

0.02682926829268293 - 0.041463414634146344im

In [10]:
aa = rand(3)
recip(aa)

MethodError: MethodError: no method matching recip(::Array{Float64,1})
Closest candidates are:
  recip(!Matched::Number) at In[5]:1

In [11]:
## recip(a::Array) = [a[i] = recip(a[i]) for i = 1:length(a)]
## OR

recip(a::Array) = map(recip,a)
recip(aa)

3-element Array{Float64,1}:
 2.999184068736088 
 1.0402102000736877
 1.224737942184426 

In [12]:
map(sin,recip(aa)) # and can map functions to the array

3-element Array{Float64,1}:
 0.14192772682436977
 0.8625106157256626 
 0.9407169834288758 

In [13]:
bb = [2.1 3.2 4.3; 9.8 8.7 7.6]

2×3 Array{Float64,2}:
 2.1  3.2  4.3
 9.8  8.7  7.6

In [14]:
recip(bb) # our definition works but returns a 2-D array

2×3 Array{Float64,2}:
 0.47619   0.3125    0.232558
 0.102041  0.114943  0.131579

In [15]:
cc = recip(aa)'.*recip(bb)  # however we can do still do this.

2×3 Array{Float64,2}:
 1.42818   0.325066  0.284823
 0.306039  0.119564  0.16115 

### Code Generation

In [16]:
incr(x) = x + 1

incr (generic function with 1 method)

In [17]:
@code_native incr(2)

	.section	__TEXT,__text,regular,pure_instructions
; Function incr {
; Location: In[16]:1
; Function +; {
; Location: In[16]:1
	decl	%eax
	leal	1(%edi), %eax
;}
	retl
	nopw	%cs:(%eax,%eax)
;}


In [18]:
@code_native incr(2.7)

	.section	__TEXT,__text,regular,pure_instructions
; Function incr {
; Location: In[16]:1
	decl	%eax
	movl	$915870136, %eax        ## imm = 0x369711B8
	addl	%eax, (%eax)
	addb	%al, (%eax)
; Function +; {
; Location: promotion.jl:313
; Function +; {
; Location: float.jl:395
	vaddsd	(%eax), %xmm0, %xmm0
;}}
	retl
	nop
;}


In [19]:
@code_native incr(2//7)

	.section	__TEXT,__text,regular,pure_instructions
; Function incr {
; Location: In[16]:1
	pushl	%ebx
	decl	%eax
	subl	$16, %esp
	decl	%eax
	movl	%edi, %ebx
	decl	%eax
	movl	$549501584, %eax        ## imm = 0x20C0BA90
	addl	%eax, (%eax)
	addb	%al, (%eax)
	decl	%eax
	movl	%esp, %edi
	movl	$1, %edx
	calll	*%eax
	vmovups	(%esp), %xmm0
	vmovups	%xmm0, (%ebx)
	decl	%eax
	movl	%ebx, %eax
	decl	%eax
	addl	$16, %esp
	popl	%ebx
	retl
	nop
;}


In [20]:
@code_native incr(2.0 + 7.0im)

	.section	__TEXT,__text,regular,pure_instructions
; Function incr {
; Location: In[16]:1
; Function +; {
; Location: complex.jl:304
; Function +; {
; Location: promotion.jl:313
; Function +; {
; Location: In[16]:1
	vmovsd	(%esi), %xmm0           ## xmm0 = mem[0],zero
	decl	%eax
	movl	$915874032, %eax        ## imm = 0x369720F0
	addl	%eax, (%eax)
	addb	%al, (%eax)
	vaddsd	(%eax), %xmm0, %xmm0
;}}
; Location: complex.jl:12
	decl	%eax
	movl	8(%esi), %eax
;}
	vmovsd	%xmm0, (%edi)
	decl	%eax
	movl	%eax, 8(%edi)
	decl	%eax
	movl	%edi, %eax
	retl
	nopw	%cs:(%eax,%eax)
;}


In [21]:
# Look at some intermediate stages
dump(:(incr(2.7)))

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: Symbol incr
    2: Float64 2.7


In [22]:
@code_lowered incr(2.7)

CodeInfo(
│1 1 ─ %1 = x + 1
│  └──      return %1
)

In [23]:
@code_typed incr(2.7)

CodeInfo(
│╻╷ +1 1 ─ %1 = (Base.add_float)(x, 1.0)::Float64
│    └──      return %1
) => Float64

In [24]:
@code_llvm(incr(2.7))


; Function incr
; Location: In[16]:1
define double @julia_incr_36859(double) {
top:
; Function +; {
; Location: promotion.jl:313
; Function +; {
; Location: float.jl:395
  %1 = fadd double %0, 1.000000e+00
;}}
  ret double %1
}


## Metaprogramming

### Symbols and Expressions

In [25]:
ex1 = :((x^2 + y^2 - 2*x*y)^0.5)

:(((x ^ 2 + y ^ 2) - 2 * x * y) ^ 0.5)

In [26]:
ex2 = quote
    (x^2 + y^2 - 2*x*y)^0.5
end

quote
    #= In[26]:2 =#
    ((x ^ 2 + y ^ 2) - 2 * x * y) ^ 0.5
end

In [27]:
# If we instantiate the variables x,y,z then we can evaluate the expression
# This is NOT a function call, so values need to be known before hand
#
x = 1.1; y = 2.5;
eval(ex1)

1.4

In [28]:
# Note that the @eval macro does NOT behave as the function call
@eval ex1

:(((x ^ 2 + y ^ 2) - 2 * x * y) ^ 0.5)

In [29]:
# Unless the expression is prefixed with '$'
@eval $ex1

# i.e.  @eval $(ex) === eval(ex)

1.4

In [30]:
# Although ex1 and ex2 are equivalent the AST of the expression 
# is slightly different
# Here it is for ex1, we leave ex2 to the reader
#
dump(ex1)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol ^
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol -
        2: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol +
            2: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol ^
                2: Symbol x
                3: Int64 2
            3: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol ^
                2: Symbol y
                3: Int64 2
        3: Expr
          head: Symbol call
          args: Array{Any}((4,))
            1: Symbol *
            2: Int64 2
            3: Symbol x
            4: Symbol y
    3: Float64 0.5


In [31]:
# Dump is very verbose, so using show_expr creates an S-expression version
#
Meta.show_sexpr(ex1)

(:call, :^, (:call, :-, (:call, :+, (:call, :^, :x, 2), (:call, :^, :y, 2)), (:call, :*, 2, :x, :y)), 0.5)

In [32]:
# Traversing a tree
#
function traverse!(ex, symbols) end

function traverse!(ex::Symbol, symbols) 
    push!(symbols, ex) 
end

function traverse!(ex::Expr, symbols)
    if ex.head == :call  # function call
        for arg in ex.args[2:end]
            traverse!(arg, symbols)  # recursive
        end
    else
        for arg in ex.args
            traverse!(arg, symbols)  # recursive
        end
    end
end

function traverse(ex::Expr) 
    symbols = Symbol[]
    traverse!(ex, symbols) 
    return unique(symbols)  # Don't output duplicate
end

traverse(ex1)

2-element Array{Symbol,1}:
 :x
 :y

### Macros

In [33]:
macro pout(n)
    if typeof(n) == Expr 
       println(n.args)
    end
    return n
end

@pout (macro with 1 method)

In [34]:
@pout x

1.1

In [35]:
@pout (x^2 + y^2 - 2*x*y)^0.5

Any[:^, :((x ^ 2 + y ^ 2) - 2 * x * y), 0.5]


1.4

In [36]:
macro dotimes(n, body)
    quote
        for i = 1:$(esc(n))
            $(esc(body))
        end
    end
end

@dotimes 3 print("Hi")

HiHiHi

In [37]:
i = 0; @dotimes 3 [global i += 1; println(i*i)]

1
4
9


### Macro Expansions

In [38]:
# Expand the @assert function
macroexpand(Main,:(@assert n > 0))

:(if n > 0
      nothing
  else
      (Base.throw)((Base.AssertionError)("n > 0"))
  end)

In [39]:
# Expand our @dotimes function, whic is somewhat simpler
#
macroexpand(Main,:(@dotimes 3 [global i += 1; println(i*i)]))

quote
    #= In[36]:3 =#
    for #10#i = 1:3
        #= In[36]:4 =#
        [global i += 1; println(i * i)]
    end
end

In [40]:
# Not all macro expansions produce short boiler plate
#
using Printf, Statistics
aa = [rand() for i = 1:100000]
@printf "The average value is %f.3 over %d trials"  mean(aa) length(aa)

The average value is 0.501538.3 over 100000 trials

In [41]:
# A more useful version of dotimes macro is until
# Creates a loop and breaks out when a condition fails
#
macro until(condition, block)
    quote
        while true
            $(esc(block))
            if $(esc(condition))
                break
            end
        end
    end
end

i = 0; @until (i >= 3) [global i += 1; println(i*i)]

1
4
9


In [42]:
# The until macro can be used to implement a simple if-then-else
#
macro iif(cond, body1, body2)
    :(if !$cond
        $(esc(body1))
    else
        $(esc(body2))
    end)
end


@iif (macro with 1 method)

In [43]:
fac(n::Integer) = (n == 1) ? 1 : n*fac(n-1)

n = 10
@iif (n < 1) fac(n) ArgumentError("$n not positive")

3628800

In [44]:
n = -1; @iif (n < 1) fac(n) ArgumentError("$n not positive")

ArgumentError("-1 not positive")

---

In [45]:
function fib(n::Integer)
  @assert n > 0
  (n == 1 || n == 2) ? 1 : fib(n-1) + fib(n-2)
end

macroexpand(Main,:(@time(fac(big(402)))))

quote
    #= util.jl:154 =#
    local #26#stats = (Base.gc_num)()
    #= util.jl:155 =#
    local #28#elapsedtime = (Base.time_ns)()
    #= util.jl:156 =#
    local #27#val = fac(big(402))
    #= util.jl:157 =#
    #28#elapsedtime = (Base.time_ns)() - #28#elapsedtime
    #= util.jl:158 =#
    local #29#diff = (Base.GC_Diff)((Base.gc_num)(), #26#stats)
    #= util.jl:159 =#
    (Base.time_print)(#28#elapsedtime, (#29#diff).allocd, (#29#diff).total_time, (Base.gc_alloc_count)(#29#diff))
    #= util.jl:161 =#
    (Base.println)()
    #= util.jl:162 =#
    #27#val
end

In [46]:
# A timing macro

# First defined a modified Kempner function
# This converts very slowly

function kempner(n::Integer)
  @assert n > 0
  s = 0.0
  r9  = r"9"
  r9x = r"9{2}"
  for i in 1:n
    if (match(r9,string(i)) == nothing) || (match(r9x,string(i)) != nothing)
      s += 1.0/float(i)
    end
  end
  return s
end

kempner (generic function with 1 method)

In [47]:
[kempner(10^i) for i in 1:7]

7-element Array{Float64,1}:
  2.8178571428571426
  4.79194977518307  
  6.629220445102462 
  8.310718044797644 
  9.850071218741334 
 11.262972084198324 
 12.563882665247327 

In [48]:
macro bmk(fex, n::Integer)
    quote 
      let s = 0.0
      if $(esc(n)) > 0
        val = $(esc(fex))
        for i = 1:$(esc(n))
          local t0 = Base.time_ns()
          local val = $(esc(fex))
          s += Base.time_ns() - t0
        end
        return s/($(esc(n)) * 10e9)
      else
        Base.error("Number of trials must be positive")
      end
    end
  end
end

@bmk (macro with 1 method)

In [49]:
@bmk kempner(10^7) 10

0.28757123785

### Horners method

In [50]:
# Evaluate a polynomial with a function
# This is NOT a macro version

function poly_native(x, a...)
  p=zero(x)
  for i = 1:length(a)
    p = p + a[i] *  x^(i-1)
  end
  return p
end

f_native(x) = poly_native(x,1,2,3,4,5)
f_native(2.1)

152.71450000000004

In [51]:
# Neither IS this!

function poly_horner(x, a...)
  b=zero(x)
  for i = length(a):-1:1
    b = a[i] + b * x
  end
  return b
end

# f -> (((5*x + 4)*x + 3)*x + 2)*x + 1
#
f_horner(x) = poly_horner(x,1,2,3,4,5)
f_horner(2.1)

152.71450000000002

In [52]:
# for the macro version we need a help function
# Define mad(x,a,b)
# [Julia has this function too :- muladd(x,a,b)]

mad(x,a,b) = a*x + b
mad(2.1,5,4)

# And NOW use Horner's method in a macro
# [ p... is a variable list of arguments passed as an array]

macro horner(x, p...)
    ex = esc(p[end])
    for i = length(p)-1:-1:1
        ex = :(mad(t, $ex, $(esc(p[i]))))
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

@horner 2.1 1 2 3 4 5

152.71450000000002

In [53]:
# The saving of eliminating the loop for a 5th order polynomial is not great
# However for larger arrays and complex calculations this can be substantial
#
macroexpand(Main,:(@horner 2.1 1 2 3 4 5))


quote
    #40#t = 2.1
    (Main.mad)(#40#t, (Main.mad)(#40#t, (Main.mad)(#40#t, (Main.mad)(#40#t, 5, 4), 3), 2), 1)
end

### MacroTools and Lazy

In [54]:
macro reverse(ex)
    if isa(ex, Expr) && ex.head == :call
        return Expr(:call, ex.args[1], reverse(ex.args[2:end])...)
    else
        return ex
    end
end

@reverse  8//11 - 5//11 * 3//11

-73//121

In [55]:
# i.e evaluates as :
3//11 * 5//11 - 8//11

-73//121

In [56]:
macro q(s)     
  s0 = eval(s)
  try
    if length(s0) > 0
      ss = split(ss,'\n')
      for i = 1:length(ss)
         println(reverse(ss[i]))
      end
    end
  catch
    return  
  end
end

@q (macro with 1 method)

---

In [57]:
# The postwalk function splits an expression into symbols and then 
# reconstructs it, so we can apply different operations to each symbol

using MacroTools:postwalk

ex = :(1 + (2 + 3) + 4)
p = postwalk(ex) do x
    x isa Integer ? fac(x) : x
end

:(1 + (2 + 6) + 24)

In [58]:
# Evaluate the expression
eval(p)

33

In [59]:
map(x -> @show(x), [1,2,3,4]);

x = 1
x = 2
x = 3
x = 4


In [60]:
postwalk(ex) do x
         @show x
end

x = :+
x = 1
x = :+
x = 2
x = 3
x = :(2 + 3)
x = 4
x = :(1 + (2 + 3) + 4)


:(1 + (2 + 3) + 4)

In [61]:
@capture(ex, a_ + b_ + c_)

true

In [62]:
b

:(2 + 3)

In [63]:
a*eval(b) + c    # => 1*5 + 4

9

In [64]:
reduce(+, 1:10)

55

In [65]:
plus(a, b) = :($a + $b)
p = reduce(plus, 1:10)

:(((((((((1 + 2) + 3) + 4) + 5) + 6) + 7) + 8) + 9) + 10)

In [66]:
eval(p)

55

In [67]:
# Do something useful
# This is the series expansion for the SINE function
# We need the factorial function and will use the version from the STDLIB
#
using SpecialFunctions
k = 2
pp = [:($((-1)^k) * x^$(1+2k) / $(factorial(1+2k))) for k = 0:5]


6-element Array{Expr,1}:
 :((1 * x ^ 1) / 1)         
 :((-1 * x ^ 3) / 6)        
 :((1 * x ^ 5) / 120)       
 :((-1 * x ^ 7) / 5040)     
 :((1 * x ^ 9) / 362880)    
 :((-1 * x ^ 11) / 39916800)

In [68]:
# We can reduce this to a single expression
reduce(plus,pp)

:((((((1 * x ^ 1) / 1 + (-1 * x ^ 3) / 6) + (1 * x ^ 5) / 120) + (-1 * x ^ 7) / 5040) + (1 * x ^ 9) / 362880) + (-1 * x ^ 11) / 39916800)

In [69]:
# ... and evaluate it for a specific value of x
x = 2.1; eval(reduce(plus,pp))

0.8632069372306019

---

In [70]:
using Lazy
import Lazy: cycle, range, drop, take

In [71]:
# Create a list of Fibonacci numbers and pick off first 20, using the @lazy macro.
#
fibs = @lazy 0:1:(fibs + drop(1, fibs));
take(20, fibs)

(0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181)

In [72]:
# Function style, pass the argument to a function
#
@> π/6 sin exp   # ==> exp(sin(π/6))

1.6487212707001282

In [73]:
# Currying: The @> can also have functional arguments
#
f(x,μ) = -(x - μ)^2
@> π/6 f(1.6) exp

0.3139129389863363

In [74]:
# The @>> macro reverse the order of the arguments
# Use this toutput the first 15 EVEN squares
#
esquares = @>> range() map(x -> x^2) filter(iseven);
take(15, esquares)

(4 16 36 64 100 144 196 256 324 400 484 576 676 784 900)

In [75]:
# Next create a list of primes
# The takewhile function is defined in Lazy
#
isprime(n) =
  @>> primes begin
    takewhile(x -> x<=sqrt(n))
    map(x -> n % x == 0)
    any; !
  end;

In [76]:
# We need to initialise the primes list
#
primes = filter(isprime, range(2));
take(20, primes)

(2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71)

In [77]:
isprime(113)

true

## Generated Functions

In [78]:
# A simple generated function to execute a multiply + add
# The result returned is a symbol
#
@generated function mad(a,b,c)
    Core.println("Calculating: a*b + c")
    return :(a * b + c)
end

mad (generic function with 1 method)

In [79]:
# Call the function
mad(2.3,1.7,1.1)

Calculating: a*b + c


5.01

In [80]:
# ... and again
# ... this time it is not re-evaluated
mad(2.3,1.7,1.1)

5.01

In [81]:
# With different types of arguments - the function is reevaluated
mad(2.3,1.7,1)

Calculating: a*b + c


4.91

In [82]:
# To illustrate the use consider the following function ...
# ... which multiplies the size of dimensions of a n-D array
#
# First a 'normal' function definition

function pdims(x::Array{T,N}) where {T,N}
  s = 1
  for i = 1:N
    s = s * size(x, i)
  end
  return s
end

pdims (generic function with 1 method)

In [83]:
# And then a 'generated function' version
@generated function gpdims(x::Array{T,N}) where {T,N}
  ex = :(1)
  for i = 1:N
     ex = :(size(x, $i) * $ex)
  end
  return ex
end

gpdims (generic function with 1 method)

In [84]:
# We need an array to test the function
aa = reshape([rand() for i = 1:1000],10,5,5,4); size(aa)

(10, 5, 5, 4)

In [85]:
# Both functions provide the same result
#
pdims(aa) == gpdims(aa)

true

In [86]:
# But if we look at the lowered code
@code_lowered pdims(aa)

CodeInfo(
│7  1 ─       s = 1
│8  │   %2  = 1:$(Expr(:static_parameter, 2))
│   │         #temp# = (Base.iterate)(%2)
│   │   %4  = #temp# === nothing
│   │   %5  = (Base.not_int)(%4)
│   └──       goto #4 if not %5
│   2 ┄ %7  = #temp#
│   │         i = (Core.getfield)(%7, 1)
│   │   %9  = (Core.getfield)(%7, 2)
│9  │   %10 = s
│   │   %11 = (Main.size)(x, i)
│   │         s = %10 * %11
│   │         #temp# = (Base.iterate)(%2, %9)
│   │   %14 = #temp# === nothing
│   │   %15 = (Base.not_int)(%14)
│   └──       goto #4 if not %15
│   3 ─       goto #2
│11 4 ─       return s
)

In [87]:
# But if we look at the lowered code
@code_lowered gpdims(aa)

CodeInfo(
│╻ macro expansion3 1 ─ %1 = (Main.size)(x, 4)
││  │   %2 = (Main.size)(x, 3)
││  │   %3 = (Main.size)(x, 2)
││  │   %4 = (Main.size)(x, 1)
││  │   %5 = %4 * 1
││  │   %6 = %3 * %5
││  │   %7 = %2 * %6
││  │   %8 = %1 * %7
││  └──      return %8
)

In [88]:
# Which therefore results in very different generated native code
@code_native pdims(aa)

	.section	__TEXT,__text,regular,pure_instructions
; Function pdims {
; Location: In[82]:7
	pushl	%eax
	decl	%eax
	movl	$4294967293, %ecx       ## imm = 0xFFFFFFFD
	movl	$1, %eax
	nopl	(%eax)
; Location: In[82]:9
; Function size; {
; Location: array.jl:154
L16:
	decl	%eax
	leal	4(%ecx), %edx
	decl	%eax
	cmpl	$4, %edx
	ja	L37
;}
; Function *; {
; Location: int.jl:54
	decl	%eax
	imull	48(%edi,%ecx,8), %eax
;}
; Function iterate; {
; Location: range.jl:575
; Function ==; {
; Location: promotion.jl:425
	decl	%eax
	testl	%ecx, %ecx
;}}
	je	L75
; Function size; {
; Location: array.jl:154
L37:
	decl	%eax
	leal	1(%ecx), %edx
	decl	%eax
	addl	$5, %ecx
	decl	%eax
	testl	%ecx, %ecx
	decl	%eax
	movl	%edx, %ecx
	jg	L16
	decl	%eax
	movl	$72430992, %eax         ## imm = 0x4513590
	addl	%eax, (%eax)
	addb	%al, (%eax)
	decl	%eax
	movl	$3399781936, %edi       ## imm = 0xCAA48E30
	sarb	%cl, (%edi)
	addb	%bh, %bh
	rcrb	-61(%ecx)
	nopl	(%eax)
;}}


In [89]:
# This version does not have the looping
@code_native gpdims(aa)

	.section	__TEXT,__text,regular,pure_instructions
; Function gpdims {
; Location: In[83]:3
; Function macro expansion; {
; Location: In[83]
; Function size; {
; Location: In[83]:3
	decl	%eax
	movl	40(%edi), %eax
;}}
; Function macro expansion; {
; Location: int.jl:54
	decl	%eax
	imull	48(%edi), %eax
	decl	%eax
	imull	32(%edi), %eax
	decl	%eax
	imull	24(%edi), %eax
;}
	retl
	nopw	%cs:(%eax,%eax)
;}


## Modularity

In [90]:
# Previous versions of Julia had an example section, absent in version v1.0
# An interesting one was modular integer, which required a little re-editing
#
module ModInts
export ModInt

import Base: +, -, *, /, inv

struct ModInt{n} <: Integer
  k::Int
  ModInt{n}(k) where n = new(mod(k,n))
end

Base.show(io::IO, k::ModInt{n}) where n =
    print(io, get(io, :compact, false) ? k.k : "$(k.k) mod $n")

(a::ModInt{n} + b::ModInt{n}) where n = ModInt{n}(a.k+b.k)
(a::ModInt{n} - b::ModInt{n}) where n = ModInt{n}(a.k-b.k)
(a::ModInt{n} * b::ModInt{n}) where n = ModInt{n}(a.k*b.k)
-(a::ModInt{n}) where n = ModInt{n}(-a.k)

inv(a::ModInt{n}) where n = ModInt{n}(invmod(a.k, n))
(a::ModInt{n} / b::ModInt{n}) where n = a*inv(b) 

Base.promote_rule(::Type{ModInt{n}}, ::Type{Int}) where n = ModInt{n}
Base.convert(::Type{ModInt{n}}, i::Int) where n = ModInt{n}(i)

end

Main.ModInts

In [91]:
# Test the module
#
using Main.ModInts
m1 = ModInt{11}(2)
m2 = ModInt{11}(7)
m3 = 3*m1 + m2  # => mod(13,11) => 2 

2 mod 11

In [92]:
# Because of multiple dispatch we can do the following
#
mm = reshape([ModInt{11}(rand(0:10)) for i = 1:100],10,10)
ma = [ModInt{11}(rand(0:10)) for i = 1:10]

mm.*ma'

10×10 Array{ModInt{11},2}:
 10  2  0   2   0  4  2  8   8  2
  5  6  9   0   9  4  6  6   9  2
  0  4  3   4   3  4  0  9   2  9
 10  1  0   0   7  1  0  9   7  6
  1  5  7  10  10  4  3  4   0  9
  6  8  1   9   5  7  0  3  10  7
  3  0  0   3   8  4  1  7   9  7
  7  9  7   9   7  3  4  5   3  9
  9  3  8   4   5  0  4  2   2  3
  6  7  3   9   0  8  6  7   6  1

---

### Ordered Pairs

In [93]:
## An Ordered Pair Module
#
module OrdPairs

import Base: +,-,*,/,==,!=,>,<,>=,<=
import Base: abs,conj,inv,zero,one,show
import LinearAlgebra: transpose,adjoint,norm

export OrdPair

struct OrdPair{T<:Number}
    a::T
    b::T
end

OrdPair(x::Number) = OrdPair(x, zero(T))

value(u::OrdPair)   = u.a
epsilon(u::OrdPair) = u.b

zero(::Type{OrdPairs.OrdPair}) = OrdPair(zero(T),zero(T))
one(::Type{OrdPairs.OrdPair})  = OrdPair(one(T),zero(T))

abs(u::OrdPair)  = abs(value(u))
norm(u::OrdPair) = norm(value(u))

+(u::OrdPair, v::OrdPair) = OrdPair(value(u) + value(v), epsilon(u) + epsilon(v))
-(u::OrdPair, v::OrdPair) = OrdPair(value(u) - value(v), epsilon(u) - epsilon(v))
*(u::OrdPair, v::OrdPair) = OrdPair(value(u)*value(v), epsilon(u)*value(v) + value(u)*epsilon(v))
/(u::OrdPair, v::OrdPair) = OrdPair(value(u)/value(v),(epsilon(u)*value(v) - value(u)*epsilon(v))/(value(v)*value(v)))

==(u::OrdPair, v::OrdPair) = norm(u) == norm(v)
!=(u::OrdPair, v::OrdPair) = norm(u) != norm(v)
>(u::OrdPair, v::OrdPair)  = norm(u) > norm(v)
>=(u::OrdPair, v::OrdPair) = norm(u) >= norm(v)
<(u::OrdPair, v::OrdPair)  = norm(u) < norm(v)
<=(u::OrdPair, v::OrdPair) = norm(u) <= norm(v)

+(x::Number, u::OrdPair) = OrdPair(value(u) + x, epsilon(u))
+(u::OrdPair, x::Number) = x + u

-(x::Number, u::OrdPair) = OrdPair(x - value(u), epsilon(u))
-(u::OrdPair, x::Number) = OrdPair(value(u) - x, epsilon(u))

*(x::Number, u::OrdPair) = OrdPair(x*value(u), x*epsilon(u))
*(u::OrdPair, x::Number) = x*u

/(u::OrdPair, x::Number) = (1.0/x)*u

conj(u::OrdPair)  = OrdPair(value(u),-epsilon(u))
inv(u::OrdPair)   = one(OrdPair)/u

transpose(u::OrdPair) = u
transpose(uu::Array{OrdPairs.OrdPair,2}) = [uu[j,i] for i=1:size(uu)[1],j=1:size(uu)[2]]
adjoint(u::OrdPair) = u

convert(::Type{OrdPair}, x::Number) = OrdPair(x,zero(x))
promote_rule(::Type{OrdPair}, ::Type{<:Number}) = OrdPair

## show(io::IO,u::OrdPair) = print(io,value(u)," + (",epsilon(u),")ϵ")

function show(io::IO,u::OrdPair) 
 op::String = (epsilon(u) < 0.0) ? " - " : " + ";
 print(io,value(u),op,abs(epsilon(u)),"ϵ")
end

end

Main.OrdPairs

In [94]:
# Exercise the module
# Notice how the show() routine provides pretty-print output

using Main.OrdPairs
p1 = OrdPair(2.3,-1.7)
p2 = OrdPair(4.4,0.9)
p1 * p2

10.12 - 5.41ϵ

In [95]:
p2/p1

1.9130434782608698 + 1.8052930056710779ϵ

In [96]:
# Again we can operate on array of Ordered Pairs

using Statistics
pp = [OrdPair(rand(),rand()) for i in 1:100]
mean(pp)

0.5069118192655038 + 0.5217286636001186ϵ

In [97]:
# And promote a rational in a mixed OP
p3 = OrdPair(2.3, 11/7)

2.3 + 1.5714285714285714ϵ

In [98]:
# The module is not a full implementation
# So this FAILS
p4 = OrdPair(2.3, 11.0 + 7.2im)

MethodError: MethodError: no method matching OrdPair(::Float64, ::Complex{Float64})
Closest candidates are:
  OrdPair(::Number) at In[93]:16
  OrdPair(::T<:Number, !Matched::T<:Number) where T<:Number at In[93]:12

In [ ]:
# Also a number of other arithmetic functions need to be imported from Base
std(p1)